In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression , Ridge , LogisticRegression
from sklearn.preprocessing import LabelEncoder , PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score , mean_squared_error
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from graphviz import Source
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix , classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")

plt.style.use('fivethirtyeight')

In [3]:
train = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/train.csv")
test = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/test.csv")

In [4]:
train.head()

In [5]:
train.shape


In [6]:
test.shape

In [7]:
train.info()

In [8]:
train.isnull().sum()

In [9]:
test.isnull().sum()

In [10]:
train.dropna(inplace=True)

In [11]:
test.dropna(inplace=True)

In [12]:
def dropds(dset):
    dset.drop(["Unnamed: 0","id"],axis=1,inplace=True)
    dset["Gender"] = dset["Gender"].map({"Male":1,"Female":0})
    dset["Customer Type"] = dset["Customer Type"].map({"Loyal Customer":1,"disloyal Customer":0})
    #travel_dum = pd.get_dummies(dset[["Type of Travel"]],drop_first=True)
    #class_dum = pd.get_dummies(dset[["Class"]],drop_first=True)


In [13]:
dropds(train)
dropds(test)

In [14]:
train.shape

In [15]:
test.shape

In [16]:
train.head()

In [17]:
train.info()

In [18]:
test.info()

In [19]:
travel_dum = pd.get_dummies(train[["Type of Travel"]],drop_first=True)
class_dum = pd.get_dummies(train[["Class"]],drop_first=True)
train = pd.concat([train,travel_dum,class_dum],axis=1)

In [20]:
train.head()

In [21]:
travel_dum = pd.get_dummies(test[["Type of Travel"]],drop_first=True)
class_dum = pd.get_dummies(test[["Class"]],drop_first=True)
test = pd.concat([test,travel_dum,class_dum],axis=1)

In [22]:
test.head()

In [23]:
train.drop(["Type of Travel","Class"],axis=1,inplace=True)

In [24]:
test.drop(["Type of Travel","Class"],axis=1,inplace=True)

In [25]:
train.head()

In [26]:
test.head()

In [27]:
train["satisfaction"] = train["satisfaction"].map({"satisfied":1,"neutral or dissatisfied":0})
test["satisfaction"] = test["satisfaction"].map({"satisfied":1,"neutral or dissatisfied":0})

In [28]:
train.head()

In [29]:
train.info()

In [30]:
import seaborn as sns 
import matplotlib.pyplot as plt 


In [31]:
plt.figure(figsize=(20,20))
#sns.pairplot(train)

In [32]:
sns.countplot(x='Online boarding',hue="satisfaction",data=train,color="green")

In [33]:
sns.histplot(x='Age',hue="satisfaction",data=train,kde=True,palette="flare")

In [34]:
sns.countplot(x='Customer Type',hue="satisfaction",data=train)

In [35]:
sns.histplot(x='Flight Distance',hue="satisfaction",data=train,kde=True,palette="dark")

In [36]:
sns.countplot(x='Inflight wifi service',hue="satisfaction",data=train,color="red")

In [37]:
sns.countplot(x='Food and drink',hue="satisfaction",data=train,color="orange")

In [38]:
plt.figure(figsize = (18,18))
sns.heatmap(train.corr(), annot = True, cmap = "RdYlGn")

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [40]:
x_train = train.drop(["satisfaction"],axis=1)
y_train = train['satisfaction']

In [41]:
x_test = test.drop(["satisfaction"],axis=1)
y_test = test["satisfaction"]


In [42]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [43]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [44]:
xgb.fit(x_train,y_train)

In [45]:
pred = xgb.predict(x_test)

In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [47]:
accuracy_score(y_test,pred)

In [48]:
confusion_matrix(y_test,pred)

In [49]:
from sklearn.ensemble import RandomForestClassifier
reg_rf = RandomForestClassifier()

In [50]:
reg_rf.fit(x_train,y_train)
#reg_rf.score(y_test,x_test)

In [51]:
rf_pred = reg_rf.predict(x_test)

In [52]:
accuracy_score(y_test,rf_pred)

In [53]:
train.columns

In [54]:
train['satisfaction'].head()

In [55]:
X=train.drop('satisfaction', axis=1)
y=train['satisfaction']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=10)

In [56]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report,precision_score, plot_roc_curve, plot_precision_recall_curve, balanced_accuracy_score

def clf_scores(clf, y_predicted):
    # Accuracy
    acc_train = clf.score(X_train, y_train)*100
    acc_test = clf.score(X_test, y_test)*100
    
    roc = roc_auc_score(y_test, y_predicted)*100 
    tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
    cm = confusion_matrix(y_test, y_predicted)
    correct = tp + tn
    incorrect = fp + fn
    d=[acc_train, acc_test,  roc, correct, incorrect,  cm]
    index=["acc_train",'Test Accuracy',"Roc Score","COrrect","Incorrect","Confusion"  ]
    output=pd.DataFrame(data=d, index=index)
    
    d=sns.heatmap(cm, annot=True)
    dd=plot_roc_curve(clf, X_train, y_train)
    ddd=plot_precision_recall_curve(clf, X_train, y_train)

    return output,d, dd, ddd

In [57]:
#1. Logistic regression

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(solver='liblinear')
clf_lr.fit(X_train, y_train)

Y_pred_lr = clf_lr.predict(X_test)
print(clf_scores(clf_lr, Y_pred_lr))

In [58]:
# 2 Random Forest

from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)

Y_pred_rf = clf_rf.predict(X_test)
print(clf_scores(clf_rf, Y_pred_rf))

In [59]:
# 3 XGboost
from sklearn.ensemble import GradientBoostingClassifier
clf_xg = GradientBoostingClassifier()
clf_xg.fit(X_train, y_train)

Y_pred_xg = clf_xg.predict(X_test)
print(clf_scores(clf_xg, Y_pred_xg))

# Fine-tune Random Forest

In [60]:
from sklearn.model_selection import GridSearchCV

param_grid = [
{'n_estimators': [10, 25], 'max_features': [5, 10], 
 'max_depth': [10, 50, None], 'bootstrap': [True, False]}
]

grid_search_forest = GridSearchCV(clf_rf, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search_forest.fit(X_train, y_train)

In [61]:
#now let's how the RMSE changes for each parameter configuration
cvres = grid_search_forest.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [62]:
#find the best model of grid search
grid_search_forest.best_estimator_

In [63]:
# Performance metrics
grid_best= grid_search_forest.best_estimator_.predict(X_train)
errors = abs(grid_best - y_train)
# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors / y_train))
# Calculate and display accuracy
accuracy = 100 - mape    
#print result
print('The best model from grid-search has an accuracy of', round(accuracy, 2),'%')

In [64]:
# Tuned Random Forest

from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_features=10, n_estimators=25)
clf_rf.fit(X_train, y_train)

Y_pred_rf = clf_rf.predict(X_test)
print(clf_scores(clf_rf, Y_pred_rf))